In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from itertools import islice
import random

# Adatbeolvasás

## Patientdata
Az eredeti csv feldolgozása már tanulásra előkészített adatokkal

In [2]:
readIn = pd.read_csv("data/df_patientdata.csv")
df_patientdata= pd.DataFrame(readIn)

In [3]:
df_patientdata.head(20)

,PatientID,Pathology,Is_Male,Age_Group_3,Age_Group_4,Age_Group_5,Age_Group_6,Age_Group_7,Age_Group_8
0,SCD0000101,1,1,False,False,True,False,False,False
1,SCD0000201,1,1,False,True,False,False,False,False
2,SCD0000301,1,1,False,False,False,False,True,False
3,SCD0000401,1,1,False,True,False,False,False,False
4,SCD0000501,1,1,False,False,False,True,False,False
5,SCD0000601,1,1,False,False,False,False,True,False
6,SCD0000701,1,1,False,True,False,False,False,False
7,SCD0000801,1,1,False,False,True,False,False,False
8,SCD0000901,1,1,False,False,False,True,False,False
9,SCD0001001,1,1,False,False,True,False,False,False


## Image data
A dicom fájlokból egységesített, normalizált képek. 
- Numpy tömbben
- Méretük: (256,256)
- Minden pixelhez csak egy érték tartozik, értékük 0-1 között van

Disclaimer!! Ez pár percig el fog tartani

In [4]:
images = np.load('data/images.npz', allow_pickle=True)

In [6]:
loaded_ids = images['ids']

In [7]:
loaded_images = images['images']

In [8]:
image_data=[]
i=0
print('start')
for id, image in zip(loaded_ids, loaded_images):
    print(f'Processing {i}', end='\r')
    image_data.append([id, image])
    i+=1

start


# Effective data loading

Amit megértettem ebből az az, hogy ahogy több órán is láthattuk, mind train-nél, mind eval-nál az adatokat batchekben kapja meg a program így könnyítve a feldolgozást. Ehhez készítettem egy sablont, ezt tudjátok használni. Ha példát szeretnétek látni a működéséről, akkor nyissátok meg a 2 - Data preparation.ipynb-t, ott több helyen használom.

A cellát duplikáljátok és az eredetit hagyjátok meg itt fenn, jól fog az még jönni :D

In [11]:
nagy_adathalmaz=[] #valami random adat itt, ez nem üres array!

feldolgozott_adat=[]

max_batch=len(nagy_adathalmaz)      #Alapesetben az adatunk hossza!
                                    #Ha csak tesztelni akarod a működést, akkor érdemes lejjebb állítani, hogy értelmes időn belül végezzen

batch_size = 50      # Válaszd meg a megfelelő batch méretet. Ha kisebb akkor nem fogja annyira terhelni a gépet, de lassabb lesz

for i in range(0, max_batch, batch_size):     #0-tól megyünk max_batch-ig, batch_size ugrásokkal
    batch = islice(nagy_adathalmaz, i, i + batch_size)     # csinálunk egy batch-nyni adatot az eredeti adatunkból az iteráció állapota alapján
    print(f'Processing {i}', end='\r')      # Opcionális sor, de türelmetleneknek (vagyis nekem) hasznos: látom hogy épp hol tart a futás
    batch_data = []     #array a batchenkénti adatnak
    for item in batch:
        
        # itt csinálsz valamit az adatokkal amik item néven futnak, pl.: tanítasz
        
        batch_data.append(item)       # összegyűjtöd az adatokat batchenként
        
    feldolgozott_adat.extend(batch_resized_images)     # belerakod batchenként az adatot a nagy tömbbe.
